# Определение части речи и начальной формы слов

В рамках большого проекта по анализу текстов вам потребовалось научиться определять части речи слов и их начальную форму на неизвестном вам языке. Сложностей добавляет то, что ваше решение должно правильно работать не только на словах, которые есть в словаре, но и на выдуманных словах, живущих по правилам этого языка (приведение в начальную форму - лемматизация - в таких случаях обычно называется псевдолемматизацией, ведь на самом деле такого слова в языке нет, как и его начальной формы).

Например, в русском языке предложение "Сяпала Калуша по напушке" не оставляет сомнений, что "сяпала" - это глагол "сяпать" в прошедшем времени (пусть такого слова и нет в словаре), а "напушке" - это существительное "напушка" в предложном падеже. Так вот вам в этот раз потребовалось научиться понимать что-то такое для неизвестного языка, да еще и без контекста, а только по самому слову.

Не имея готовый инструмент для лемматизации, но имея небольшой словарик начальных форм разных слов, вы решили не разбираться с грамматикой загадочного языка и просто быстро обучить какую-нибудь простую модель с помощью машинного обучения. 

Вам предстоит не только извлечь признаки и построить модель, но и поставить задачу. В этот раз вам решать, как свести задачу лемматизации и определения части речи к каким-то стандартным задачам машинного обучения, и от этого решения будет зависеть, какого качества получится достичь.

В качестве небольшого упрощения в задании оставлены только три части речи.

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

train = pd.read_csv("task2_lemmas_train.csv", error_bad_lines=False)

b'Skipping line 102: expected 3 fields, saw 4\nSkipping line 229: expected 3 fields, saw 4\nSkipping line 297: expected 3 fields, saw 4\nSkipping line 373: expected 3 fields, saw 4\nSkipping line 399: expected 3 fields, saw 4\nSkipping line 410: expected 3 fields, saw 4\nSkipping line 414: expected 3 fields, saw 4\nSkipping line 488: expected 3 fields, saw 4\nSkipping line 501: expected 3 fields, saw 4\nSkipping line 508: expected 3 fields, saw 4\nSkipping line 563: expected 3 fields, saw 4\nSkipping line 622: expected 3 fields, saw 4\nSkipping line 712: expected 3 fields, saw 4\nSkipping line 751: expected 3 fields, saw 4\nSkipping line 787: expected 3 fields, saw 4\nSkipping line 804: expected 3 fields, saw 4\nSkipping line 825: expected 3 fields, saw 4\nSkipping line 886: expected 3 fields, saw 4\nSkipping line 919: expected 3 fields, saw 4\nSkipping line 1043: expected 3 fields, saw 4\nSkipping line 1109: expected 3 fields, saw 4\nSkipping line 1141: expected 3 fields, saw 4\nSkipp

Для начала посмотрим на данные.

In [2]:
y_column = [w.split('+') for w in list(train.y)]
train['form'] = pd.Series(w[0] for w in y_column)    
train['type'] = pd.Series(w[1] for w in y_column)   

In [3]:
train.head(5)

,Id,X,y,form,type
0,1,vergognerete,vergognare+V,vergognare,V
1,2,amnistiavate,amnistiare+V,amnistiare,V
2,3,menomazione,menomazione+N,menomazione,N
3,4,sfaldavamo,sfaldare+V,sfaldare,V
4,5,sfodererei,sfoderare+V,sfoderare,V


In [24]:
words_train = train.X
words_form = train.form
words_type = train.type

In [10]:
print(len(words_train))

116477


In [4]:
test = pd.read_csv("task2_lemmas_test.csv", error_bad_lines=False)
test.head(5)

,Id,X
0,1,gettonan
1,2,incidentali
2,3,involtino
3,4,lievi
4,5,comunistizzasse


In [7]:
words_test = test.X

Идея решения простая (и не использующая машинное обучение). Train data будем использовать как словарь. Для каждого слова из тестовой выборки будем искать наиболее похожее слово из train. Наиболее похожее слово будем искать так: это то слово, которое имеет со словом из теста наибольший общий префикс. Когда наиболее похожее слово найдено, то для слова из теста отвечаем начальной формой и частью речи для этого похожего слова.

In [17]:
def length_of_common_prefix(s1,s2):
    l = min(len(s1), len(s2))
    prefix_len = 0
    for i in range(l):
        if s1[i] == s2[i]:
            prefix_len += 1
        else:
            break
    return prefix_len

In [21]:
print(length_of_common_prefix('happy', 'hat'))

2


In [40]:
predictions = []

# for each word in test we will look through whole words from train and find word with maximum common prefix

for test_word in words_test:
    counter = 0         # id of train word 
    id_of_similar_word = 0
    max_length_of_prefix = 0
    for train_word in words_train:
        counter += 1
        cur_length = length_of_common_prefix(test_word, train_word)
        if cur_length > max_length_of_prefix:
            max_length_of_prefix = cur_length
            id_of_similar_word = counter
    predictions.append([words_form[id_of_similar_word], words_type[id_of_similar_word]])

In [33]:
print(predictions[:10])

[['svezzare', 'V'], ['compartimentare', 'A'], ['scandaletto', 'N'], ['pattuizione', 'N'], ['aggettivare', 'V'], ['cancellare', 'V'], ['vertere', 'V'], ['accorgere', 'V'], ['cubare', 'V'], ['scontroso', 'A']]


Это все. Для каждого слова получили начальную форму и часть речи. Записываем ответ.

In [32]:
def write_answer(i, preds):
    word = preds[i][0]
    type_of_word = preds[i][1]
    return word+type_of_word

In [33]:
test['Category'] = [write_answer(i, predictions) for i in range(len(test))]
test = test.drop('X', axis = 1)
test.to_csv("subm.txt", sep=',', index=False)